In [15]:
import os
import random
from collections import defaultdict
import numpy as np
import copy
import re

In [76]:
os.environ['PATH'] = '/opt/anaconda3/bin:/opt/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin'

In [77]:
import pickle

In [78]:
#load in dictionary of normal variation with allele freq > .001

os.chdir('/Users/beth/Documents/SNV project/')
with open('gnomad_dictionary_2.pickle','rb') as read_file:
    normal_var_dict=pickle.load(read_file)


In [89]:
#create the functions for base change, insertion, deletion
def update_sequence(changes_list, seq, offset, start, chrom, current_organ, ref_seq):
    
    #if list doesn't contain any changes just return the same seqs and offset
    if len(changes_list) == 1 and changes_list[0] == "skip":
        
        return seq, ref_seq, offset
            
    
    else:
        #loop over changes list
        for i in range(len(changes_list)):
            
            #do nothing if it is a skip
            if changes_list[i] == "skip":
                continue

            #if it's a base change do this:
            #(all these cases are so it works with TCGA, gnomAD and GENIE format)
            if changes_list[i][2] != "-" and \
            changes_list[i][4] != "-" and \
            changes_list[i][3] != "-" and \
            ((len(changes_list[i][2]) == len(changes_list[i][3]) and isinstance(changes_list[i][4],float)==True) or\
            (len(changes_list[i][2]) == len(changes_list[i][3])) and \
            (len(changes_list[i][2]) == len(str(changes_list[i][4])) and isinstance(changes_list[i][4],float) == False)) :
                
                #print('base change')
                if changes_list[i][2]!=changes_list[i][3]:
                    new_base = changes_list[i][3]
                else:
                    new_base = changes_list[i][4]
                ref_pos = int(changes_list[i][1]) -1
                adjust = int(offset[ref_pos - start])

                seq = seq[0:ref_pos + adjust - start] + new_base + seq[ref_pos + adjust - start + len(new_base) -1 + 1: ]

                ref_seq = ref_seq
                
            #if it's an insertion do this:
            elif changes_list[i][2] == "-" or ( len(changes_list[i][2]) < len(str(changes_list[i][4])) \
                                               and isinstance(changes_list[i][4],float) == False):
                
                #print('insertion')
                if changes_list[i][2]!=changes_list[i][3]:
                    new_base = changes_list[i][3]
                else:
                    new_base = changes_list[i][4]
    
                    
                ref_pos = int(changes_list[i][1]) -1
                adjust = int(offset[ref_pos - start])
                
                if changes_list[i][2] == "-":
                    expected_base = ""
                    genie_fix=0 
                else:
                    expected_base = changes_list[i][2]
                    genie_fix = 1

                seq = seq[0:ref_pos + adjust - start +1 - genie_fix] + new_base + seq[ref_pos + adjust - start +1 : 100-len(new_base) +genie_fix]

                if len(seq) > 100:
                    seq = seq[:100]
                    
                #note that ref_seqs over 100bp were truncated later in processing
                ref_seq = ref_seq[0:ref_pos + adjust - start +1 - genie_fix] + "-" * (len(new_base)-len(expected_base)) + ref_seq[ref_pos + adjust - start +1 : 100-len(new_base) +genie_fix]
                
                #the offset keeps track of the positions relative to reference after the insertion 
                offset= np.add(offset,  np.concatenate((np.zeros(ref_pos-start+1-genie_fix),np.repeat(len(new_base)-len(expected_base), start+100-ref_pos-1+genie_fix))))
                
                
            #if it's a deletion do this:
            elif changes_list[i][4]=="-" or changes_list[i][3]=="-" or \
            (len(str(changes_list[i][4])) < len(changes_list[i][2]) and \
             isinstance(changes_list[i][4],float) == False) :
                
                #print('deletion')
                if changes_list[i][4] != "-" and changes_list[i][3] != "-":
                    new_base = changes_list[i][4]
                    len_del = len(str(changes_list[i][2])) - len(str(changes_list[i][4]))
                    genie_fix = 1
                else:
                    new_base = ""
                    len_del = len(str(changes_list[i][2]))
                    genie_fix = 0
                
                ref_pos = int(changes_list[i][1]) -1
                adjust = int(offset[ref_pos - start])
                expected_base = changes_list[i][2]
                

                seq = seq[0:ref_pos + adjust - start] + new_base + "-" * len_del + seq[ref_pos + adjust -start + len(expected_base) : 100] 

                ref_seq = ref_seq
                
                #note that subsequent changes in the deletion range will be added on top of the deletion
                #this behavior should be changed in the next iteration of this project


                
        return seq, ref_seq, offset
            

In [72]:
def generate_sequences(TCGA_file,samples,current_organ,normal_var_dict):
    #looping over each sample
    for i in range(95,len(samples)):
        current_sample = samples[i]
        sample_mutations = !grep $current_sample $TCGA_file | tail -n +2 | cut -f5,6,11,12,13
        sample_mutations = list(map(lambda x: tuple(x.split('\t')), sample_mutations))
        sample_mutations_dict = defaultdict(list)

        #creating sample_mutations_dict
        for i in range(len(sample_mutations)):
            sample_mutations_dict[sample_mutations[i][0]].append([int(sample_mutations[i][1]), sample_mutations[i][2], sample_mutations[i][3], sample_mutations[i][4]])

        window_starts=[]
        skip_list=[]
        #looping over each mutation in sample_mutations list of tuples 
        for i in range(len(sample_mutations)):
            current_location = sample_mutations[i]

            #to get windows of 100bp around it (note: hg38 is one position behind both MAF and VCF)
            len_mutation = max(len(current_location[2]),len(current_location[4]))
            if len_mutation > 45:
                skip_list.append(current_location)
                window_starts.append('skip')
                continue
            for j in range(1):
                window = range(int(current_location[1])-100+len_mutation,int(current_location[1])-len_mutation)
                start = random.choice(list(window))
                end = start + 100
                chrom = current_location[0]
                window_starts.append(start)
                with open("sample_locations_{}.tmp".format(current_organ), "a") as file:
                    file.write(chrom + "\t" + str(start) + "\t" + str(end) + "\n")
        
        locations = "sample_locations_{}.tmp".format(current_organ)
        #getting the hg38 sequences for the windows     
        #lock.acquire()
        ref_seqs = !bedtools getfasta -fi /Volumes/BethMac/hg38/hg38.fa -bed $locations
        #lock.release()

        #loop once for each window
        for i in range(1,len(ref_seqs),2):
            #getting the seq
            current_location=sample_mutations[i//2]
            if current_location in skip_list:
                continue
            ref_seq= ref_seqs[i]
            chrom = current_location[0]
            seq=copy.copy(ref_seq)
            start=window_starts[i//2]

            #getting a list of tuples for each mutation within the window
            nearby_mutations=[current_location]
            for i in range(len(sample_mutations_dict[chrom])):
                x = int(sample_mutations_dict[chrom][i][0])
                if x > int(start) and x < (int(start) + 100) and x != int(current_location[1]):
                    nearby_mutations.append(sample_mutations_dict[chrom][i])

            #add chrom names to mutation lists
            for i in range(1,len(nearby_mutations)):
                nearby_mutations[i] = [chrom]+nearby_mutations[i]

            #getting a list of tuples for possible normal variation within the window
            nearby_variation=[]
            for i in range(len(normal_var_dict[chrom])):
                x = int(normal_var_dict[chrom][i][0])
                if x > start and x < (start + 100):
                    nearby_variation.append(normal_var_dict[chrom][i])

            #add chrom names to normal variation lists
            for i in range(len(nearby_variation)):
                nearby_variation[i] = [chrom]+nearby_variation[i]

            #selecting which normal variation to include for the window
            nearby_variation_random=[] 
            for i in range(len(nearby_variation)):
                freq = nearby_variation[i][4]
                if freq > 0.5:
                    included = random.choices([nearby_variation[i], "skip"], weights=[freq, 1-freq], k=1)
                else:
                    included = random.choices([nearby_variation[i], "skip"], weights=[0.5, 0.5], k=1)
                nearby_variation_random.append(included[0])

            #make changes and append sequence to files

            #the offset keeps track of effects of indels
            offset=np.zeros(100).astype(int)

            #add in the nearby normal variation
            for i in range(len(nearby_variation_random)):
                seq, ref_seq, offset = update_sequence(nearby_variation_random, seq, offset, start, chrom, current_organ, ref_seq)

            #if a big insertion leads to other variation being outside the 100bp window those changes will 
            #append sequence to the end and can be removed
            if len(seq)>100:
                seq=seq[:100]

            with open('{}_normal.csv'.format(current_organ), 'a') as file:
                file.write(current_sample + "," + seq + "," + ref_seq + "\n")

            #add in the mutations on top of normal variation
            for i in range(len(nearby_mutations)):
                seq, ref_seq, offset = update_sequence(nearby_mutations, seq, offset, start, chrom, current_organ, ref_seq)

            #if big insertion made a mutation was outside of the 100bp window it will have
            #appeneded to the end. So there may be no mutation in the 100bp window and it should
            #be discarded if over 100bp
            if len(seq)==100:
                with open('{}_tumor.csv'.format(current_organ), 'a') as file:
                    file.write(current_sample + "," + seq + "," + ref_seq + "\n")
        
        #clear the sample locations file
        
        !rm $locations





In [81]:
#get the samples list from the TCGA MAF file - here liver cancer
os.chdir("/Users/beth/Desktop/MetisProject5data_2")
TCGA_file_liver = "TCGA.LIHC.mutect.a630f0a0-39b3-4aab-8181-89c1dde8d3e2.DR-10.0.somatic.maf"

samples_liver = !tail -n +7 $TCGA_file_liver | cut -f16 | sort | uniq

In [90]:
os.getcwd()

'/Users/beth/Desktop/MetisProject5data_2'

In [91]:
generate_sequences(TCGA_file_liver,samples_liver,'liver',normal_var_dict)

In [92]:
TCGA_file_bladder = "TCGA.BLCA.mutect.0e239d8f-47b0-4e47-9716-e9ecc87605b9.DR-10.0.somatic.maf"
samples_bladder = !tail -n +7 $TCGA_file_bladder | cut -f16 | sort | uniq

generate_sequences(TCGA_file_bladder,samples_bladder,'bladder',normal_var_dict)

In [93]:
TCGA_file_breast = "TCGA.BRCA.mutect.995c0111-d90b-4140-bee7-3845436c3b42.DR-10.0.somatic.maf"
samples_breast = !tail -n +7 $TCGA_file_breast | cut -f16 | sort | uniq

generate_sequences(TCGA_file_breast,samples_breast,'breast',normal_var_dict)

In [94]:
TCGA_file_colorectal = "TCGA.COAD.mutect.03652df4-6090-4f5a-a2ff-ee28a37f9301.DR-10.0.somatic.maf"
samples_colorectal = !tail -n +7 $TCGA_file_colorectal | cut -f16 | sort | uniq

generate_sequences(TCGA_file_colorectal,samples_colorectal,'colorectal',normal_var_dict)

In [95]:
TCGA_file_glioma = "TCGA.LGG.mutect.1e0694ca-fcde-41d3-9ae3-47cfaf527f25.DR-10.0.somatic.maf"
samples_glioma = !tail -n +7 $TCGA_file_glioma | cut -f16 | sort | uniq

generate_sequences(TCGA_file_glioma,samples_glioma,'glioma',normal_var_dict)

In [96]:
TCGA_file_glioblastoma = "TCGA.GBM.mutect.da904cd3-79d7-4ae3-b6c0-e7127998b3e6.DR-10.0.somatic.maf"
samples_glioblastoma = !tail -n +7 $TCGA_file_glioblastoma | cut -f16 | sort | uniq

generate_sequences(TCGA_file_glioblastoma,samples_glioblastoma,'glioblastoma',normal_var_dict)

In [97]:
TCGA_file_renal = "TCGA.KIRC.mutect.2a8f2c83-8b5e-4987-8dbf-01f7ee24dc26.DR-10.0.somatic.maf"
samples_renal = !tail -n +7 $TCGA_file_renal | cut -f16 | sort | uniq

generate_sequences(TCGA_file_renal,samples_renal,'renal',normal_var_dict)

In [98]:
TCGA_file_lung = "TCGA.LUAD.mutect.0458c57f-316c-4a7c-9294-ccd11c97c2f9.DR-10.0.somatic.maf"
samples_lung = !tail -n +7 $TCGA_file_lung | cut -f16 | sort | uniq

generate_sequences(TCGA_file_lung,samples_lung,'lung',normal_var_dict)

In [99]:
TCGA_file_pancreatic = "TCGA.PAAD.mutect.fea333b5-78e0-43c8-bf76-4c78dd3fac92.DR-10.0.somatic.maf"
samples_pancreatic = !tail -n +7 $TCGA_file_pancreatic | cut -f16 | sort | uniq

generate_sequences(TCGA_file_pancreatic,samples_pancreatic,'pancreatic',normal_var_dict)

In [100]:
TCGA_file_prostate = "TCGA.PRAD.mutect.deca36be-bf05-441a-b2e4-394228f23fbe.DR-10.0.somatic.maf"
samples_prostate = !tail -n +7 $TCGA_file_prostate | cut -f16 | sort | uniq

generate_sequences(TCGA_file_prostate,samples_prostate,'prostate',normal_var_dict)

In [101]:
TCGA_file_skin = "TCGA.SKCM.mutect.4b7a5729-b83e-4837-9b61-a6002dce1c0a.DR-10.0.somatic.maf"
samples_skin = !tail -n +7 $TCGA_file_skin | cut -f16 | sort | uniq

generate_sequences(TCGA_file_skin,samples_skin,'skin',normal_var_dict)

In [102]:
TCGA_file_stomach = "TCGA.STAD.mutect.c06465a3-50e7-46f7-b2dd-7bd654ca206b.DR-10.0.somatic.maf"
samples_stomach = !tail -n +7 $TCGA_file_stomach | cut -f16 | sort | uniq

generate_sequences(TCGA_file_stomach,samples_stomach,'stomach',normal_var_dict)

In [103]:
TCGA_file_uterine = "TCGA.UCEC.mutect.d3fa70be-520a-420e-bb6d-651aeee5cb50.DR-10.0.somatic.maf"
samples_uterine = !tail -n +7 $TCGA_file_uterine | cut -f16 | sort | uniq

generate_sequences(TCGA_file_uterine,samples_uterine,'uterine',normal_var_dict)

In [24]:
def generate_sequences_genie(genie_file,samples,normal_var_dict):
    #looping over each sample
    for i in range(len(samples)):
        current_sample = samples[i][0]
        current_organ = samples[i][1]
        if "/" in current_organ:
            current_organ = re.search("^([A-Z]*)/", current_organ)[1]
        #sample_mutations = !grep $current_sample $TCGA_file | tail -n +2 | cut -f5,6,11,12,13
        sample_mutations = !grep $current_sample $genie_file | cut -f2,3,4,5,6 -d ","
        #sample_mutations = list(map(lambda x: tuple(x.split('\t')), sample_mutations))
        sample_mutations = list(map(lambda x: tuple(x.split(',')), sample_mutations))
        sample_mutations_dict = defaultdict(list)

        #creating sample_mutations_dict
        for i in range(len(sample_mutations)):
            sample_mutations_dict[sample_mutations[i][0]].append([int(sample_mutations[i][1]), sample_mutations[i][2], sample_mutations[i][3], sample_mutations[i][4]])

        window_starts=[]
        skip_list=[]
        #looping over each mutation in sample_mutations list of tuples 
        for i in range(len(sample_mutations)):
            current_location = sample_mutations[i]

            #to get windows of 100bp around it (note: hg38 is one position behind both MAF and VCF)
            len_mutation = max(len(current_location[2]),len(current_location[4]))
            if len_mutation > 45:
                skip_list.append(current_location)
                window_starts.append('skip')
                continue
            for j in range(1):
                window = range(int(current_location[1])-100+len_mutation,int(current_location[1])-len_mutation)
                start = random.choice(list(window))
                end = start + 100
                chrom = current_location[0]
                window_starts.append(start)
                with open("sample_locations_{}.tmp".format(current_organ), "a") as file:
                    file.write(chrom + "\t" + str(start) + "\t" + str(end) + "\n")
        
        locations = "sample_locations_{}.tmp".format(current_organ)
        #getting the hg38 sequences for the windows     
        #lock.acquire()
        ref_seqs = !bedtools getfasta -fi /Volumes/BethMac/hg38/hg38.fa -bed $locations
        #lock.release()

        #loop once for each window
        for i in range(1,len(ref_seqs),2):
            #getting the seq
            current_location=sample_mutations[i//2]
            if current_location in skip_list:
                continue
            ref_seq= ref_seqs[i]
            chrom = current_location[0]
            seq=copy.copy(ref_seq)
            start=window_starts[i//2]

            #getting a list of tuples for each mutation within the window
            nearby_mutations=[current_location]
            for i in range(len(sample_mutations_dict[chrom])):
                x = int(sample_mutations_dict[chrom][i][0])
                if x > int(start) and x < (int(start) + 100) and x != int(current_location[1]):
                    nearby_mutations.append(sample_mutations_dict[chrom][i])

            #add chrom names to mutation lists
            for i in range(1,len(nearby_mutations)):
                nearby_mutations[i] = [chrom]+nearby_mutations[i]

            #getting a list of tuples for possible normal variation within the window
            nearby_variation=[]
            for i in range(len(normal_var_dict[chrom])):
                x = int(normal_var_dict[chrom][i][0])
                if x > start and x < (start + 100):
                    nearby_variation.append(normal_var_dict[chrom][i])

            #add chrom names to normal variation lists
            for i in range(len(nearby_variation)):
                nearby_variation[i] = [chrom]+nearby_variation[i]

            #selecting which normal variation to include for the window
            nearby_variation_random=[] 
            for i in range(len(nearby_variation)):
                freq = nearby_variation[i][4]
                if freq > 0.5:
                    included = random.choices([nearby_variation[i], "skip"], weights=[freq, 1-freq], k=1)
                else:
                    included = random.choices([nearby_variation[i], "skip"], weights=[0.5, 0.5], k=1)
                nearby_variation_random.append(included[0])

            #make changes and append sequence to files

            #the offset keeps track of effects of indels
            offset=np.zeros(100).astype(int)

            #add in the nearby normal variation
            for i in range(len(nearby_variation_random)):
                seq, offset = update_sequence(nearby_variation_random, seq, offset, start, chrom, current_organ)

            #if a big insertion leads to other variation outside the 100bp window those will 
            #append to the end and can be removed
            if len(seq)>100:
                seq=seq[:100]

            with open('{}_normal.csv'.format(current_organ), 'a') as file:
                file.write(current_sample + "," + seq + "," + ref_seq + "\n")

            #add in the mutations on top of normal variation
            for i in range(len(nearby_mutations)):
                seq, offset = update_sequence(nearby_mutations, seq, offset, start, chrom, current_organ)

            #if big insertion made a mutation was outside of the 100bp window it will have
            #appeneded to the end. So there may be no mutation in the 100bp window and it should
            #be discarded if over 100bp
            if len(seq)==100:
                with open('{}_tumor.csv'.format(current_organ), 'a') as file:
                    file.write(current_sample + "," + seq + "," + ref_seq + "\n")
        
        #clear the sample locations file
        
        !rm $locations




### test zone

In [49]:
changes_list=[['chr1', 151198956, '-', '-', 'x']]

In [50]:
changes_list[0]

['chr1', 151198956, '-', '-', 'x']

In [63]:
chrom = changes_list[0][0]
start = changes_list[0][1] - 10
end = start + 100

In [64]:
import os
os.chdir('/Users/beth/Desktop/MetisProject5data_2/')

In [65]:
!rm temp_location.bed

In [66]:
with open('temp_location.bed', 'w') as file:
    file.write(chrom + "\t" + str(start) + "\t" + str(end))
additional_sequence = !bedtools getfasta -fi /Volumes/BethMac/hg38/hg38.fa -bed temp_location.bed

In [67]:
seq = additional_sequence[1]

In [68]:
seq

'GAACCGGATTGAAAGAGAGCCAGGCCGCTGAGGGGGAGGGGGCTGCTAAGATGGCGTCGGCCTCCTCCGGGCCGTCGTCTTCGGTCGGTTTTTCATCCTT'

In [69]:
ref_seq = copy.copy(seq)

In [70]:
offset=np.zeros(100).astype(int)

In [71]:
update_sequence(changes_list, seq, offset, start, chrom, 'test', ref_seq)

insertion
GAACCGGATTxGAAAGAGAGCCAGGCCGCTGAGGGGGAGGGGGCTGCTAAGATGGCGTCGGCCTCCTCCGGGCCGTCGTCTTCGGTCGGTTTTTCATCCT
GAACCGGATT-GAAAGAGAGCCAGGCCGCTGAGGGGGAGGGGGCTGCTAAGATGGCGTCGGCCTCCTCCGGGCCGTCGTCTTCGGTCGGTTTTTCATCCT
151198955
151198946
0
1
0


('GAACCGGATTxGAAAGAGAGCCAGGCCGCTGAGGGGGAGGGGGCTGCTAAGATGGCGTCGGCCTCCTCCGGGCCGTCGTCTTCGGTCGGTTTTTCATCCT',
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))